In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from ppo import PPO
from config_utils import get_config
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import utils

In [ ]:
from envs import env_carracing_simple
env = env_carracing_simple.CarRacingSimple()

In [ ]:
# config = get_config("configs/config-2020_06_02.json")
config = get_config("configs/config-2020_06_07-resnet18.json")

# config.actor_critic["actor"]["learning_rate"] = 0.0001
# config.actor_critic["critic"]["learning_rate"] = 0.0001
print(config)

num_epochs = 1500
# num_epochs = 100
# num_epochs = 2
minibatch_size = 1

if config.has_training:
    num_epochs = config.training.get("num_epochs", num_epochs)
    minibatch_size = config.training.get("minibatch_size", minibatch_size)

In [ ]:
checkpoint_interval = 10
experiment_dir = "experiments/2020_06_07-01-resnet18"

os.makedirs(experiment_dir, exist_ok=True)

In [ ]:
from actor_critics import carracing_resnet_18_ac as ac
actor = ac.Actor(env)
critic = ac.Critic(env)

ppo = PPO(
    env,
    config=config,
    actor=actor,
    critic=critic,
    no_cuda=False
)

In [ ]:
loss_vals = []
pbar = tqdm(range(num_epochs))
for i in pbar:
    step_loss_vals = ppo.step()
    loss_vals.append(sum(step_loss_vals) / len(step_loss_vals))
    pbar.set_description("Loss = {:.6f}".format(loss_vals[-1]))
    
    if i%checkpoint_interval == 0:
        utils.save_ppo_checkpoint(
            exp_dir = experiment_dir,
            epoch = i,
            loss = loss_vals[-1],
            ppo = ppo,
        )

In [ ]:
plt.plot(loss_vals)
plt.xlabel("step")
plt.ylabel("loss")
plt.title("Training Loss Curve")
plt.show()

In [ ]:
import torch
from torch.autograd import Variable

In [ ]:
#env_new = wrappers.Monitor(env, './video/', force = True)
times = 0
env_new = env
observation = env_new.reset()
# observation = Variable(torch.from_numpy(observation), requires_grad=False).to(ppo.torch_device)
observation = torch.unsqueeze(
    torch.from_numpy(observation.copy()),
    0
).to(ppo.torch_device)
for _ in range(100):
    env_new.render()
    action = torch.argmax(ppo.pi(observation.float())) 
    observation, reward, done, info = env_new.step(action.cpu().detach().numpy())
    # observation = Variable(torch.from_numpy(observation), requires_grad=False).to(ppo.torch_device)
    observation = torch.unsqueeze(
        torch.from_numpy(observation.copy()),
        0
    ).to(ppo.torch_device)
    if not done:
        times += 1
        
print(times)
env_new.close()